In [11]:
#import libraries

import pandas as pd
import numpy as np
import seaborn as sb
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)

from sklearn.cross_validation import train_test_split
from pandas import Series, DataFrame
from pylab import rcParams


from sklearn.metrics import classification_report,accuracy_score
sb.set(style="white")
sb.set(style="whitegrid", color_codes=True)
import itertools
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV,StratifiedKFold, LeaveOneOut
from sklearn.datasets import make_regression

import warnings; warnings.simplefilter('ignore')

In [12]:
### reading data and printing the data ###
data = pd.read_csv('Normalized_Consolidated_List.csv')
print(data.shape)
print(list(data.columns))

(171, 13)
['ID', 'TruncatedID', 'Mean', 'SD', 'AUC', 'Peak', 'MaxIndex', 'Halfpk', 'Skew', 'HHMean', 'Fat', 'Carb', 'Protein']


In [13]:
### CHO
CHO_data = data.drop(['ID','Protein','Fat'], axis=1) #taking only the Carb target column
print(CHO_data.Carb.unique())# Finding the unique values in that column
print(CHO_data.TruncatedID.unique()) #finiding unique IDs

[51 48 54]
['2133-002' '2133-022' '2133-020' '2133-004' '2133-010' '2133-015'
 '2133-040' '2133-013' '2133-008' '2133-019' '2133-001' '2133-024'
 '2133-011' '2133-018' '2133-006' '2133-032' '2133-025' '2133-017'
 '2133-028' '2133-039' '2133-021' '2133-030' '2133-033' '2133-037'
 '2133-026' '2133-012' '2133-036' '2133-035' '2133-041' '2133-009']


In [14]:
###for LOW-HIGH

CHO_LH_data = CHO_data[CHO_data['Carb'].isin([48,54])] #considering the two classes for binary classification
CHO_LH_data.loc[CHO_LH_data.Carb == 48, 'CHO_bin'] = 0 #giving them binary values, as double variables sometimes create error
CHO_LH_data.loc[CHO_LH_data.Carb== 54, 'CHO_bin'] = 1
CHO_LH_data = CHO_LH_data.drop(['Carb'], axis=1)
#the above 4 lines will be different for different classification, the remaining code will be same
X = CHO_LH_data.loc[:, CHO_LH_data.columns != 'CHO_bin'] #Feature set
Y = CHO_LH_data.loc[:, CHO_LH_data.columns == 'CHO_bin'] #target value/label

values =CHO_data.TruncatedID.unique() #needed for conducting LOSO

final_accuracy =[] #will be used for finding mean across cross validation

for i in range(0,30): #as we have 30 IDs
    print("For fold :", i+1) #current fold number
    test =CHO_LH_data[CHO_LH_data['TruncatedID'].isin([values[i]])] #test data    
    train =CHO_LH_data[~CHO_LH_data['TruncatedID'].isin([values[i]])]#train data
    X_train = train.loc[:, train.columns != 'CHO_bin'] #test feature
    Y_train = train.loc[:, train.columns == 'CHO_bin'] #test label
    X_test = test.loc[:, test.columns != 'CHO_bin']#train feature    
    Y_test =test.loc[:, test.columns == 'CHO_bin']#train label
    
    train_ids = X_train['TruncatedID']#needed for inner cross-validation
    train_ids = np.array(train_ids)#needed for inner cross-validation
    train_ids = np.unique(train_ids)#needed for inner cross-validation
    
    X_train = X_train.drop(['TruncatedID'],axis = 1) #dropping this column as it is not a feature for classification
    X_test = X_test.drop(['TruncatedID'],axis = 1)#dropping this column as it is not a feature for classification 
    
    #print("len of train", len(train))
    max_acc = 0 #initializing maximum accuracy which will be needed later    
    ypred_all = []
    
    for C_val in range(1,11): #range of number of neighbours
        ypred_all = []
        for j in range(0,29): #as inner cross validation ha 29 IDs
            inner_test = train[train['TruncatedID'].isin([train_ids[j]])] #validation set
            inner_train = train[~train['TruncatedID'].isin([train_ids[j]])]#inner train set
            xtr = inner_train.loc[:, inner_train.columns != 'CHO_bin'] #inner train set features
            ytr = inner_train.loc[:, inner_train.columns == 'CHO_bin']# inner train set label
            xvl = inner_test.loc[:, inner_test.columns != 'CHO_bin']#validation set features
            yvl = inner_test.loc[:, inner_test.columns == 'CHO_bin']#validation set labels
            xtr = xtr.drop(['TruncatedID'],axis = 1)#We dont need this column for classification        
            xvl = xvl.drop(['TruncatedID'],axis = 1)#We dont need this column for classification
            clf = KNeighborsClassifier(n_neighbors=C_val) #KNN classifier

            clf=clf.fit(xtr,ytr)
            ypred = clf.predict(xvl)#prediction
            ypred_all.append(accuracy_score(yvl,ypred))
        ypred_mean = np.mean(ypred_all)#finding mean of all accuracy
        if max_acc<ypred_mean:#finding the depth which gives maximum accuracy
            max_acc = ypred_mean
            best_param = C_val
    
    #best KNN classifier
    print("best neighbours",best_param)
    knn = KNeighborsClassifier(n_neighbors=best_param)
    knn.fit(X_train,Y_train)
    Y_pred = knn.predict(X_test)
    final_accuracy.append(accuracy_score(Y_test,Y_pred))
    print("Accuracy score : ", accuracy_score(Y_test,Y_pred))
print("The final accuracy is : ", np.mean(final_accuracy))#final result (mean of accuracies)

For fold : 1
best neighbours 8
Accuracy score :  0.6666666666666666
For fold : 2
best neighbours 8
Accuracy score :  0.25
For fold : 3
best neighbours 10
Accuracy score :  0.75
For fold : 4
best neighbours 10
Accuracy score :  0.5
For fold : 5
best neighbours 8
Accuracy score :  0.5
For fold : 6
best neighbours 10
Accuracy score :  0.25
For fold : 7
best neighbours 8
Accuracy score :  0.5
For fold : 8
best neighbours 8
Accuracy score :  0.75
For fold : 9
best neighbours 8
Accuracy score :  0.5
For fold : 10
best neighbours 8
Accuracy score :  0.75
For fold : 11
best neighbours 8
Accuracy score :  0.5
For fold : 12
best neighbours 8
Accuracy score :  0.75
For fold : 13
best neighbours 10
Accuracy score :  0.75
For fold : 14
best neighbours 10
Accuracy score :  0.5
For fold : 15
best neighbours 8
Accuracy score :  0.5
For fold : 16
best neighbours 8
Accuracy score :  0.5
For fold : 17
best neighbours 8
Accuracy score :  0.5
For fold : 18
best neighbours 8
Accuracy score :  0.75
For fold 

In [15]:
#MEDIUM-HIGH

CHO_LH_data = CHO_data[CHO_data['Carb'].isin([51,54])] #considering the two classes for binary classification
CHO_LH_data.loc[CHO_LH_data.Carb == 51, 'CHO_bin'] = 0 #giving them binary values, as double variables sometimes create error
CHO_LH_data.loc[CHO_LH_data.Carb== 54, 'CHO_bin'] = 1
CHO_LH_data = CHO_LH_data.drop(['Carb'], axis=1)
#the above 4 lines will be different for different classification, the remaining code will be same
X = CHO_LH_data.loc[:, CHO_LH_data.columns != 'CHO_bin'] #Feature set
Y = CHO_LH_data.loc[:, CHO_LH_data.columns == 'CHO_bin'] #target value/label

values =CHO_data.TruncatedID.unique() #needed for conducting LOSO

final_accuracy =[] #will be used for finding mean across cross validation

for i in range(0,30): #as we have 30 IDs
    print("For fold :", i+1) #current fold number
    test =CHO_LH_data[CHO_LH_data['TruncatedID'].isin([values[i]])] #test data    
    train =CHO_LH_data[~CHO_LH_data['TruncatedID'].isin([values[i]])]#train data
    X_train = train.loc[:, train.columns != 'CHO_bin'] #test feature
    Y_train = train.loc[:, train.columns == 'CHO_bin'] #test label
    X_test = test.loc[:, test.columns != 'CHO_bin']#train feature    
    Y_test =test.loc[:, test.columns == 'CHO_bin']#train label
    
    train_ids = X_train['TruncatedID']#needed for inner cross-validation
    train_ids = np.array(train_ids)#needed for inner cross-validation
    train_ids = np.unique(train_ids)#needed for inner cross-validation
    
    X_train = X_train.drop(['TruncatedID'],axis = 1) #dropping this column as it is not a feature for classification
    X_test = X_test.drop(['TruncatedID'],axis = 1)#dropping this column as it is not a feature for classification 
    
    #print("len of train", len(train))
    max_acc = 0 #initializing maximum accuracy which will be needed later    
    ypred_all = []
    
    for C_val in range(1,11): #range of number of neighbours
        ypred_all = []
        for j in range(0,29): #as inner cross validation ha 29 IDs
            inner_test = train[train['TruncatedID'].isin([train_ids[j]])] #validation set
            inner_train = train[~train['TruncatedID'].isin([train_ids[j]])]#inner train set
            xtr = inner_train.loc[:, inner_train.columns != 'CHO_bin'] #inner train set features
            ytr = inner_train.loc[:, inner_train.columns == 'CHO_bin']# inner train set label
            xvl = inner_test.loc[:, inner_test.columns != 'CHO_bin']#validation set features
            yvl = inner_test.loc[:, inner_test.columns == 'CHO_bin']#validation set labels
            xtr = xtr.drop(['TruncatedID'],axis = 1)#We dont need this column for classification        
            xvl = xvl.drop(['TruncatedID'],axis = 1)#We dont need this column for classification
            clf = KNeighborsClassifier(n_neighbors=C_val) #KNN classifier

            clf=clf.fit(xtr,ytr)
            ypred = clf.predict(xvl)#prediction
            ypred_all.append(accuracy_score(yvl,ypred))
        ypred_mean = np.mean(ypred_all)#finding mean of all accuracy
        if max_acc<ypred_mean:#finding the depth which gives maximum accuracy
            max_acc = ypred_mean
            best_param = C_val
    
    #best KNN classifier
    print("best neighbours",best_param)
    knn = KNeighborsClassifier(n_neighbors=best_param)
    knn.fit(X_train,Y_train)
    Y_pred = knn.predict(X_test)
    final_accuracy.append(accuracy_score(Y_test,Y_pred))
    print("Accuracy score : ", accuracy_score(Y_test,Y_pred))
print("The final accuracy is : ", np.mean(final_accuracy))#final result (mean of accuracies)

For fold : 1
best neighbours 6
Accuracy score :  0.75
For fold : 2
best neighbours 3
Accuracy score :  0.25
For fold : 3
best neighbours 5
Accuracy score :  1.0
For fold : 4
best neighbours 7
Accuracy score :  0.75
For fold : 5
best neighbours 3
Accuracy score :  0.75
For fold : 6
best neighbours 7
Accuracy score :  1.0
For fold : 7
best neighbours 3
Accuracy score :  0.75
For fold : 8
best neighbours 7
Accuracy score :  0.75
For fold : 9
best neighbours 3
Accuracy score :  1.0
For fold : 10
best neighbours 7
Accuracy score :  1.0
For fold : 11
best neighbours 10
Accuracy score :  0.75
For fold : 12
best neighbours 7
Accuracy score :  0.75
For fold : 13
best neighbours 3
Accuracy score :  1.0
For fold : 14
best neighbours 3
Accuracy score :  0.6666666666666666
For fold : 15
best neighbours 3
Accuracy score :  0.75
For fold : 16
best neighbours 3
Accuracy score :  0.5
For fold : 17
best neighbours 3
Accuracy score :  0.75
For fold : 18
best neighbours 7
Accuracy score :  0.3333333333333

In [16]:
#LOW-MEDIUM

CHO_LH_data = CHO_data[CHO_data['Carb'].isin([51,48])] #considering the two classes for binary classification
CHO_LH_data.loc[CHO_LH_data.Carb == 51, 'CHO_bin'] = 0 #giving them binary values, as double variables sometimes create error
CHO_LH_data.loc[CHO_LH_data.Carb== 48, 'CHO_bin'] = 1
CHO_LH_data = CHO_LH_data.drop(['Carb'], axis=1)
#the above 4 lines will be different for different classification, the remaining code will be same
X = CHO_LH_data.loc[:, CHO_LH_data.columns != 'CHO_bin'] #Feature set
Y = CHO_LH_data.loc[:, CHO_LH_data.columns == 'CHO_bin'] #target value/label

values =CHO_data.TruncatedID.unique() #needed for conducting LOSO

final_accuracy =[] #will be used for finding mean across cross validation

for i in range(0,30): #as we have 30 IDs
    print("For fold :", i+1) #current fold number
    test =CHO_LH_data[CHO_LH_data['TruncatedID'].isin([values[i]])] #test data    
    train =CHO_LH_data[~CHO_LH_data['TruncatedID'].isin([values[i]])]#train data
    X_train = train.loc[:, train.columns != 'CHO_bin'] #test feature
    Y_train = train.loc[:, train.columns == 'CHO_bin'] #test label
    X_test = test.loc[:, test.columns != 'CHO_bin']#train feature    
    Y_test =test.loc[:, test.columns == 'CHO_bin']#train label
    
    train_ids = X_train['TruncatedID']#needed for inner cross-validation
    train_ids = np.array(train_ids)#needed for inner cross-validation
    train_ids = np.unique(train_ids)#needed for inner cross-validation
    
    X_train = X_train.drop(['TruncatedID'],axis = 1) #dropping this column as it is not a feature for classification
    X_test = X_test.drop(['TruncatedID'],axis = 1)#dropping this column as it is not a feature for classification 
    
    #print("len of train", len(train))
    max_acc = 0 #initializing maximum accuracy which will be needed later    
    ypred_all = []
    
    for C_val in range(1,11): #range of number of neighbours
        ypred_all = []
        for j in range(0,29): #as inner cross validation ha 29 IDs
            inner_test = train[train['TruncatedID'].isin([train_ids[j]])] #validation set
            inner_train = train[~train['TruncatedID'].isin([train_ids[j]])]#inner train set
            xtr = inner_train.loc[:, inner_train.columns != 'CHO_bin'] #inner train set features
            ytr = inner_train.loc[:, inner_train.columns == 'CHO_bin']# inner train set label
            xvl = inner_test.loc[:, inner_test.columns != 'CHO_bin']#validation set features
            yvl = inner_test.loc[:, inner_test.columns == 'CHO_bin']#validation set labels
            xtr = xtr.drop(['TruncatedID'],axis = 1)#We dont need this column for classification        
            xvl = xvl.drop(['TruncatedID'],axis = 1)#We dont need this column for classification
            clf = KNeighborsClassifier(n_neighbors=C_val) #KNN classifier

            clf=clf.fit(xtr,ytr)
            ypred = clf.predict(xvl)#prediction
            ypred_all.append(accuracy_score(yvl,ypred))
        ypred_mean = np.mean(ypred_all)#finding mean of all accuracy
        if max_acc<ypred_mean:#finding the depth which gives maximum accuracy
            max_acc = ypred_mean
            best_param = C_val
    
    #best KNN classifier
    print("best neighbours",best_param)
    knn = KNeighborsClassifier(n_neighbors=best_param)
    knn.fit(X_train,Y_train)
    Y_pred = knn.predict(X_test)
    final_accuracy.append(accuracy_score(Y_test,Y_pred))
    print("Accuracy score : ", accuracy_score(Y_test,Y_pred))
print("The final accuracy is : ", np.mean(final_accuracy))#final result (mean of accuracies)

For fold : 1
best neighbours 6
Accuracy score :  0.6666666666666666
For fold : 2
best neighbours 8
Accuracy score :  0.5
For fold : 3
best neighbours 8
Accuracy score :  0.75
For fold : 4
best neighbours 4
Accuracy score :  0.5
For fold : 5
best neighbours 4
Accuracy score :  0.75
For fold : 6
best neighbours 4
Accuracy score :  0.25
For fold : 7
best neighbours 8
Accuracy score :  0.75
For fold : 8
best neighbours 8
Accuracy score :  0.75
For fold : 9
best neighbours 4
Accuracy score :  0.75
For fold : 10
best neighbours 4
Accuracy score :  1.0
For fold : 11
best neighbours 6
Accuracy score :  1.0
For fold : 12
best neighbours 8
Accuracy score :  0.25
For fold : 13
best neighbours 8
Accuracy score :  0.75
For fold : 14
best neighbours 8
Accuracy score :  0.6666666666666666
For fold : 15
best neighbours 8
Accuracy score :  0.75
For fold : 16
best neighbours 8
Accuracy score :  0.5
For fold : 17
best neighbours 8
Accuracy score :  0.75
For fold : 18
best neighbours 7
Accuracy score :  0